In [3]:
import deepchem as dc
from deepchem.feat.graph_data import GraphData
from deepchem.feat import MolGraphConvFeaturizer
import numpy as np
import pandas as pd

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normalization for NumSpiroAtoms. Feature removed!
No normalization for NumUnspecifiedAtomStereoCenters. Feature removed!
No normalization for Phi. Feature removed!
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (C:\Users\yyyyx\miniconda3\envs\deepchem\lib\site-packages\deepchem\models\torch_models\__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [4]:
df = pd.read_csv('./HIV_protease_BBB.csv')

In [5]:
df

,Name,Structure,BBB penetration
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,High
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,High
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,High
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,High
...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Low
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Low
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Low
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,High


In [6]:
model = dc.models.GCNModel(
    n_tasks=1,
    graph_conv_layers=[128, 128],
    dense_layer_size=256,
    dropout=0.2,
    mode='classification',
    learning_rate=0.0005,
    batch_size=128,
)

In [7]:
model.restore('./BBB_penetration/model_1st/checkpoint1.pt')

In [8]:
model

In [9]:
# 1. SMILES에서 그래프 형식으로 변환
featurizer = MolGraphConvFeaturizer()
features = featurizer.featurize(df["Structure"])  # Structure 컬럼에서 특징 추출

In [10]:
# 2. 레이블 설정 (pIC50)
labels = None

In [11]:
# 3. 가중치 (필요시 기본값으로 1 설정)
weights = None  # 기본적으로 None으로 설정. 커스텀 가중치가 있으면 지정.

# 4. 데이터셋 생성
dataset = dc.data.NumpyDataset(X=features, y=labels, w=weights)

In [12]:
# 데이터 크기 확인
print(f"Feature shape: {features[0].node_features.shape}")


Feature shape: (46, 30)


In [13]:
# 예측 수행
y_pred_prob = model.predict(dataset)

In [14]:
# 가장 큰 값을 가진 인덱스를 클래스로 결정
y_pred = np.argmax(y_pred_prob, axis=1)

In [15]:
y_pred_prob

array([[0.08380944, 0.91619056],
       [0.40097263, 0.5990274 ],
       [0.00313607, 0.99686396],
       [0.34085584, 0.65914416],
       [0.4640906 , 0.5359094 ],
       [0.3952961 , 0.6047039 ],
       [0.6340657 , 0.36593434],
       [0.56455296, 0.43544698],
       [0.9787133 , 0.02128678],
       [0.60070264, 0.39929742],
       [0.47435972, 0.52564025],
       [0.3099316 , 0.69006836],
       [0.13139041, 0.86860967],
       [0.5373989 , 0.46260113],
       [0.20498742, 0.7950126 ],
       [0.23551632, 0.76448375],
       [0.2966722 , 0.70332783],
       [0.74901456, 0.2509854 ],
       [0.78041476, 0.21958524],
       [0.29667222, 0.7033278 ],
       [0.49781466, 0.5021854 ],
       [0.8340543 , 0.16594574],
       [0.2406072 , 0.7593928 ],
       [0.32301408, 0.6769859 ],
       [0.8936713 , 0.10632877],
       [0.28250676, 0.71749324],
       [0.27111822, 0.7288817 ],
       [0.2697885 , 0.73021156],
       [0.56053644, 0.43946356],
       [0.40349823, 0.59650177],
       [0.

In [16]:
# 예측 수행
df['Prediction'] = y_pred

In [17]:
df

,Name,Structure,BBB penetration,Prediction
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,High,1
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,High,1
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,High,1
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN,1
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,High,1
...,...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Low,0
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Low,1
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Low,1
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,High,1


In [18]:
df['Prediction'].sum()

60

In [19]:
df.to_csv('./HIV114_BBB_prediction.csv')

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# StandardScaler를 사용해 Prediction 값을 원복
# mean = scaler.mean_[0]  # StandardScaler에서 사용된 평균
# std = scaler.scale_[0]  # StandardScaler에서 사용된 표준편차

# 원복 공식 적용
df["Original Prediction"] = df["Prediction"] * 2.0954617919366605 -3.0520097604259093

# 결과 확인
print(df.head())


                               Name  \
0               CHEMBL729/LOPINAVIR   
1              CHEMBL584/NELFINAVIR   
2               CHEMBL57/NEVIRAPINE   
3  CHEMBL282042/SAQUINAVIR MESYLATE   
4           CHEMBL222559/TIPRANAVIR   

                                           Structure Solubility  Prediction  \
0  CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...    Soluble   -4.426832   
1  CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...   Moderate   -4.294361   
2              CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1   Moderate   -3.793959   
3  CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...        NaN   -2.228862   
4  CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...  Insoluble   -4.859251   

   Original Prediction  
0           -12.328266  
1           -12.050678  
2           -11.002106  
3            -7.722506  
4           -13.234385  


In [23]:
mean = scaler.mean_[0]
std = scaler.scale_[0]

AttributeError: 'StandardScaler' object has no attribute 'mean_'

In [25]:
df

,Name,Structure,Solubility,Prediction,Original Prediction
0,CHEMBL729/LOPINAVIR,CC1=CC=CC(C)=C1OCC(=O)N[C@@H](CC1=CC=CC=C1)[C@...,Soluble,-5.241626,-14.035636
1,CHEMBL584/NELFINAVIR,CC1=C(O)C=CC=C1C(=O)N[C@@H](CSC1=CC=CC=C1)[C@H...,Moderate,-5.525194,-14.629843
2,CHEMBL57/NEVIRAPINE,CC1=CC=NC2=C1NC(=O)C1=CC=CN=C1N2C1CC1,Moderate,-3.309443,-9.986820
3,CHEMBL282042/SAQUINAVIR MESYLATE,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,NaN,-5.772190,-15.147413
4,CHEMBL222559/TIPRANAVIR,CCC[C@@]1(CCC2=CC=CC=C2)CC(O)=C([C@H](CC)C2=CC...,Insoluble,-7.282715,-18.312662
...,...,...,...,...,...
108,CHEMBL100089,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2CCCC[C@@H]2CN1C[C...,Moderate,-6.030370,-15.688419
109,CHEMBL100048,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Soluble,-3.489471,-10.364064
110,CHEMBL100040,CC(C)(C)NC(=O)[C@@H]1C[C@@H]2SCC[C@@H]2CN1C[C@...,Moderate,-5.755380,-15.112189
111,CHEMBL100039,CC(C)CCN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)[C...,Soluble,-4.615815,-12.724273
